<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
import requests
import pandas as pd

In [0]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [30]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...


In [31]:
df_toc['text'][0]

'ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nContents\r\n\r\nACT I\r\nScene I. Rossillon. A room in the Countess’s palace.\r\nScene II. Paris. A room in the King’s palace.\r\nScene III. Rossillon. A Room in the Palace.\r\n\r\n\r\nACT II\r\nScene I. Paris. A room in the King’s palace.\r\nScene II. Rossillon. A room in the Countess’s palace.\r\nScene III. Paris. The King’s palace.\r\nScene IV. Paris. The King’s palace.\r\nScene V. Another room in the same.\r\n\r\n\r\nACT III\r\nScene I. Florence. A room in the Duke’s palace.\r\nScene II. Rossillon. A room in the Countess’s palace.\r\nScene III. Florence. Before the Duke’s palace.\r\nScene IV. Rossillon. A room in the Countess’s palace.\r\nScene V. Without the walls of Florence.\r\nScene VI. Camp before Florence.\r\nScene VII. Florence. A room in the Widow’s house.\r\n\r\n\r\nACT IV\r\nScene I. Without the Florentine camp.\r\nScene II. Florence. A room in the Widow’s house.\r\nScene III. The Florentine camp.\r\nScene IV. Florence. A room in 

In [0]:
def wrangle_text(x):
  rows = []
  for row in x['text']:
    row = row.replace("\r", "")
    row = row.replace("\n", " ")
    row = row.strip()
    rows.append(row)
  x['clean_text'] = rows

  trims = []
  for trim in x['clean_text']:
    trim = trim[1990:]
    trims.append(trim)
  x['trimmed_text'] = trims
  # x['trimmed_text'] = x['clean_text'][20][1990:]
  return x

In [53]:
clean_df = df_toc.copy()
clean_df = wrangle_text(clean_df)
clean_df.head()

,title,start,end,text,clean_text,trimmed_text
0,ALL’S WELL THAT ENDS WELL,2777,7738,ALL’S WELL THAT ENDS WELL\r\n\r\n\r\n\r\nConte...,ALL’S WELL THAT ENDS WELL Contents ACT I S...,"S. In delivering my son from me, I bury a seco..."
1,THE TRAGEDY OF ANTONY AND CLEOPATRA,7739,11840,THE TRAGEDY OF ANTONY AND CLEOPATRA\r\n\r\nDRA...,THE TRAGEDY OF ANTONY AND CLEOPATRA DRAMATIS ...,world transform'd Into a strumpet's fool....
2,AS YOU LIKE IT,11841,14631,AS YOU LIKE IT\r\n\r\nDRAMATIS PERSONAE.\r\n\r...,"AS YOU LIKE IT DRAMATIS PERSONAE. DUKE, li...","it, Adam, that grieves me; and the spirit of..."
3,THE COMEDY OF ERRORS,14632,17832,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...,THE COMEDY OF ERRORS Contents ACT I Scene ...,amount unto a hundred marks; Therefore by law...
4,THE TRAGEDY OF CORIOLANUS,17833,27806,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...,THE TRAGEDY OF CORIOLANUS Dramatis Personae ...,thirst for revenge. SECOND CITIZEN. Would...


In [54]:
clean_df.shape

(43, 6)

In [55]:
# Kept getting particularly strage resutls, noticed the outputs
# were seeming to delete themselves as they were typing out, often leaving
# very few, or no characters/sequences. Generated text also had lots of spaces.
# Dug into it and realized much of the predicted text was "\r\n", and I learned
# that "\r" actually removes the preceeding text from what I could tell.
# Started from beginning again to try and have cleaner text to work with.

len(clean_df['trimmed_text'][0])#[150:]

132627

In [56]:
clean_df['trimmed_text'][20]

' myself.  SALARINO. Your mind is tossing on the ocean, There where your argosies, with portly sail Like signiors and rich burghers on the flood, Or as it were the pageants of the sea, Do overpeer the petty traffickers That curtsy to them, do them reverence, As they fly by them with their woven wings.  SOLANIO. Believe me, sir, had I such venture forth, The better part of my affections would Be with my hopes abroad. I should be still Plucking the grass to know where sits the wind, Peering in maps for ports, and piers and roads; And every object that might make me fear Misfortune to my ventures, out of doubt Would make me sad.  SALARINO. My wind cooling my broth Would blow me to an ague when I thought What harm a wind too great might do at sea. I should not see the sandy hour-glass run But I should think of shallows and of flats, And see my wealthy Andrew dock’d in sand, Vailing her high top lower than her ribs To kiss her burial. Should I go to church And see the holy edifice of stone 

In [0]:
### Import tools

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


### Encode Data as characters

In [0]:
data = clean_df['trimmed_text'].tolist()

In [59]:
data[0]

'S. In delivering my son from me, I bury a second husband.  BERTRAM. And I in going, madam, weep o’er my father’s death anew; but I must attend his majesty’s command, to whom I am now in ward, evermore in subjection.  LAFEW. You shall find of the king a husband, madam; you, sir, a father. He that so generally is at all times good, must of necessity hold his virtue to you, whose worthiness would stir it up where it wanted, rather than lack it where there is such abundance.  COUNTESS. What hope is there of his majesty’s amendment?  LAFEW. He hath abandon’d his physicians, madam; under whose practices he hath persecuted time with hope, and finds no other advantage in the process but only the losing of hope by time.  COUNTESS. This young gentlewoman had a father—O that “had!”, how sad a passage ’tis!—whose skill was almost as great as his honesty; had it stretch’d so far, would have made nature immortal, and death should have play for lack of work. Would for the king’s sake he were living!

In [60]:
# Gather all text

text = " ".join(data)

chars = list(set(text))

# Lookup tables
char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"This corpus contains {len(chars)} unique characters")

This corpus contains 104 unique characters


In [0]:
# text[:150]

In [0]:
# len(char_int)

In [0]:
# int_char[105]

In [0]:
# print('{')
# for char,_ in zip(char_int, range(20)):
#     print('  {:4s}: {:3d},'.format(repr(char), char_int[char]))
# print('  ...\n}')

In [64]:
# Create the sequence data

maxlen = 40
step = 10

encoded = [char_int[c] for c in text]

sequences = [] # each element should be 30 characters long
next_chars = []

for i in range(0, len(encoded) - maxlen, step):
  sequences.append(encoded[i : i + maxlen])
  next_chars.append(encoded[i + maxlen])

print('sequences:', len(sequences))

sequences: 538775


In [0]:
import numpy as np

# Specify x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    x[i,t,char] = 1

  y[i, next_chars[i]] = 1

In [66]:
x.shape

(538775, 40, 104)

In [0]:
# print(x[0][0])
# print(sequences[0][0])

In [67]:
y.shape

(538775, 104)

### Modeling

In [0]:
# Build the model: single LSTM

"""
activation == tanh
recurrent_activation == sigmoid
recurrent_dropout == 0
unroll is False
use_bias is True
Inputs are not masked or strictly right padded.
"""

model = Sequential()
model.add(LSTM(1028, 
               input_shape=(maxlen, len(chars)), 
              #  dropout=0.2, 
              #  unroll=False,
              #  use_bias=True
               ))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1028)              4667120   
_________________________________________________________________
dense (Dense)                (None, 106)               109074    
Total params: 4,776,194
Trainable params: 4,776,194
Non-trainable params: 0
_________________________________________________________________


In [0]:
import random

def sample(preds):
  """Helper function to sample index from probability array"""
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / 1
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
import sys

def on_epoch_end(epoch, _):
  """Function invoked at end of each epoch to print generated text"""

  print()
  print("----- Generating text after Epoch: %d" % epoch)

  start_index = random.randint(0, len(text) - maxlen - 1)

  generated = ""

  sentence = text[start_index: start_index + maxlen]
  generated += sentence

  print('----- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)

  for i in range (400): 
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_int[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = int_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
# Fit the model

# model.fit(x,
#           y,
#           batch_size=1024,
#           # validation_split=.2,
#           epochs=15,
#           callbacks=[print_callback])

Epoch 1/15
1101/1101 [==============================] - ETA: 0s - loss: 2.3880
----- Generating text after Epoch: 0
----- Generating with seed: "ls. They would have me as fami"











    Hee hos, te werch wh thile dreth th m his are worl fornmed becofrety'd and Su
1101/1101 [==============================] - 169s 153ms/step - loss: 2.3880
Epoch 2/15
1101/1101 [==============================] - ETA: 0s - loss: 1.8126
----- Generating text after Epoch: 1
----- Generating with seed: " pass by, pluck Casca by the s"








Excunwly and with berom gove; which de is
1101/1101 [==============================] - 169s 153ms/step - loss: 1.8126
Epoch 3/15
1101/1101 [==============================] - ETA: 0s - loss: 1.6106
----- Generating text after Epoch: 2
----- Generating with seed: "ent to give him good
    repo"
ent to give him good










    He brot all stopl
1101/1101 [==============================] - 169s 153ms/step - loss: 1.6106
Epoch 4/15
1101/1101 [========================

### Model II

In [0]:
model2 = Sequential()
model2.add(LSTM(1028, 
               input_shape=(maxlen, len(chars))
               ))
model2.add(Dense(len(chars), activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='nadam')

In [69]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1028)              4658896   
_________________________________________________________________
dense_1 (Dense)              (None, 104)               107016    
Total params: 4,765,912
Trainable params: 4,765,912
Non-trainable params: 0
_________________________________________________________________


In [0]:
import random

def sample(preds):
  """Helper function to sample index from probability array"""
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / 1
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
import sys

def on_epoch_end(epoch, _):
  """Function invoked at end of each epoch to print generated text"""

  print()
  print("----- Generating text after Epoch: %d" % epoch)

  start_index = random.randint(0, len(text) - maxlen - 1)

  generated = ""

  sentence = text[start_index: start_index + maxlen]
  generated += sentence

  print('----- Generating with seed: "' + sentence + '"')
  sys.stdout.write(generated)

  for i in range (400): 
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_int[char]] = 1

    preds = model2.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = int_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
model2.fit(x,
           y,
           batch_size=1024,
           epochs=15,
           callbacks=[print_callback])

Epoch 1/15
526/527 [============================>.] - ETA: 0s - loss: 2.1490
----- Generating text after Epoch: 0
----- Generating with seed: "URREY. Has the King this?   SUFFOLK. Bel"
URREY. Has the King this?   SUFFOLK. Belwer sor farere. The but or Merveris THONS] NOMALA. I  p        r Wix O CEmVars. ]  Sor, Whrightm gos, wirt had._] ]by, Card Proud._]  TENTRUKIX.        Ant urad O soR.   RONCOND NORTLANN]   aDi', Alr to rarto, I  usur, Sis not shes, diy torns. Why low and merist wit thary her and, txe Dupler’n torer.  CEr I Jres Ry Sart THam APLEROLEY, Lib OD_] Werrestr, ond ant.t thal, in gllees, Mave the sromt, 
527/527 [==============================] - 109s 208ms/step - loss: 2.1490
Epoch 2/15
526/527 [============================>.] - ETA: 0s - loss: 1.9159
----- Generating text after Epoch: 1
----- Generating with seed: " her in.                       [Exit BAS"
 her in.                       [Exit BASTISE      friens and to bariby you loke with ond gobe pico,     Ay ateclle:

In [0]:
test_preds = model2.predict(x, verbose=1)

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN